### Import libraries

In [13]:
try:
    import pandas as pd
except ImportError:
    !pip install pandas
    import pandas as pd

try:
    import numpy as np
except ImportError:
    !pip install numpy
    import numpy as np

try:
    from surprise import Reader, Dataset, SVD
except ImportError:
    !pip install scikit-surprise
    from surprise import Reader, Dataset, SVD


### Load & Preprocess data

In [ ]:

# base_movies = pd.read_csv('datasets/base-movies-dataset/movies.csv')
# base_ratings = pd.read_csv('datasets/base-movies-dataset/ratings.csv')
# base_tags = pd.read_csv('datasets/base-movies-dataset/tags.csv')
# base_links = pd.read_csv('datasets/base-movies-dataset/links.csv')

# print(f'Number rows of movies: {len(base_movies)}')
# print(f'Number rows of ratings: {len(base_ratings)}')
# print(f'Number rows of tags: {len(base_tags)}')
# print(f'Number rows of links: {len(base_links)}')

Number rows of movies: 87585
Number rows of ratings: 32000204
Number rows of tags: 2000072
Number rows of links: 87585


In [ ]:
# movies = base_movies
# movies.to_csv('datasets/adapted-movies-dataset/movies.csv', index=False)

# ratings = pd.read_csv('datasets/base-movies-dataset/ratings.csv', nrows=87585)
# ratings.to_csv('datasets/adapted-movies-dataset/ratings.csv', index=False)

# tags = pd.read_csv('datasets/base-movies-dataset/tags.csv', nrows=87585)
# tags.to_csv('datasets/adapted-movies-dataset/tags.csv', index=False)

# links = base_links
# links.to_csv('datasets/adapted-movies-dataset/links.csv', index=False)

In [42]:
movies = pd.read_csv('datasets/movies-1/adapted-movies-dataset/movies.csv')
ratings = pd.read_csv('datasets/movies-1/adapted-movies-dataset/ratings.csv')
tags = pd.read_csv('datasets/movies-1/adapted-movies-dataset/tags.csv')
links = pd.read_csv('datasets/movies-1/adapted-movies-dataset/links.csv')

In [43]:
print(f'Number rows of movies: {len(movies)}')
print(f'Number rows of ratings: {len(ratings)}')
print(f'Number rows of tags: {len(tags)}')
print(f'Number rows of links: {len(links)}')

Number rows of movies: 87585
Number rows of ratings: 87585
Number rows of tags: 87585
Number rows of links: 87585


In [17]:
print(f'Movies columns: {movies.columns}')
print(f'Movies columns: {ratings.columns}')

Movies columns: Index(['movieId', 'title', 'genres'], dtype='object')
Movies columns: Index(['userId', 'movieId', 'rating', 'timestamp'], dtype='object')


In [18]:

movies_ratings = ratings.merge(movies, on='movieId', how='left')
movies_ratings.head(6)




,userId,movieId,rating,timestamp,title,genres
0,1,17,4.0,944249077,Sense and Sensibility (1995),Drama|Romance
1,1,25,1.0,944250228,Leaving Las Vegas (1995),Drama|Romance
2,1,29,2.0,943230976,"City of Lost Children, The (Cité des enfants p...",Adventure|Drama|Fantasy|Mystery|Sci-Fi
3,1,30,5.0,944249077,Shanghai Triad (Yao a yao yao dao waipo qiao) ...,Crime|Drama
4,1,32,5.0,943228858,Twelve Monkeys (a.k.a. 12 Monkeys) (1995),Mystery|Sci-Fi|Thriller
5,1,34,2.0,943228491,Babe (1995),Children|Drama


In [38]:
#tags_grouped = tags.groupby(['userId', 'movieId'])['tag'].apply(lambda x: ', '.join(x)).reset_index()

# tags['tag'] = tags['tag'].fillna('').astype(str)
# tags_grouped = tags.groupby(['userId', 'movieId'])['tag'].apply(lambda x: ', '.join(x)).reset_index()
# df = tags_grouped.merge(tags_grouped, on=['userId', 'movieId'], how="left")
# df.head(6)

In [40]:
# ratings_count = df.groupby('userId').size()
# active_users = ratings_count[ratings_count >= 20].index
# df_filtered = df[df['userId'].isin(active_users)]

# print(f"Usuarios activos: {len(active_users)}")
# print(f"Total de registros después del filtrado: {df_filtered.shape[0]}")

In [39]:
# df_filtered.to_csv("datasets/movies-dataset/total_movies_cleaned.csv", index=False)

In [23]:
ratings.head()

,userId,movieId,rating,timestamp
0,1,17,4.0,944249077
1,1,25,1.0,944250228
2,1,29,2.0,943230976
3,1,30,5.0,944249077
4,1,32,5.0,943228858


In [29]:
from surprise.model_selection import train_test_split, cross_validate

In [26]:
reader = Reader()
data = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)
trainset, testset = train_test_split(data, test_size=0.2)


In [30]:
svd = SVD()
results = cross_validate(svd, data, cv=5, measures=['RMSE', 'MAE'], verbose=True)

# Mostrar los resultados promedio
print("RMSE promedio:", results['test_rmse'].mean())
print("MAE promedio:", results['test_mae'].mean())

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9162  0.9094  0.9199  0.9131  0.9095  0.9136  0.0040  
MAE (testset)     0.7051  0.6971  0.7040  0.7026  0.7015  0.7021  0.0027  
Fit time          2.91    2.90    2.94    2.91    2.90    2.91    0.01    
Test time         0.10    0.09    0.09    0.09    0.09    0.09    0.01    
RMSE promedio: 0.9136057609766528
MAE promedio: 0.702053435379687


In [31]:
trainset = data.build_full_trainset()
svd.fit(trainset)

In [33]:
ratings[ratings['userId'] == 1]

,userId,movieId,rating,timestamp
0,1,17,4.0,944249077
1,1,25,1.0,944250228
2,1,29,2.0,943230976
3,1,30,5.0,944249077
4,1,32,5.0,943228858
...,...,...,...,...
136,1,2985,5.0,943230976
137,1,2997,4.0,943226846
138,1,3030,4.0,943228400
139,1,3078,2.0,943226986


In [ ]:
svd.predict(1, 29, 3)

Prediction(uid=1, iid=25, r_ui=3, est=2.50782970395725, details={'was_impossible': False})